# Linear Solver Performance: Constant Coefficient Poisson, MPI-parallel

### Part 1, Benchmark Setup and Execution

In [1]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

The below script needs to be able to find the current output cell; this is an easy method to get it.

Execution Date/time is 23.08.2022 12:10:31


In [2]:
#r "BoSSSpad.dll"
//#r "../../../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [3]:
using BoSSS.Application.SipPoisson;

In [4]:
string PROJECT_NAME = System.Environment.GetEnvironmentVariable("LinslvPerfPar") ?? "LinslvPerfPar"; // this allows to modify the project name for testing purposes
wmg.Init(PROJECT_NAME);
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

Project name is set to 'WIP-k-LinslvPerfPar'.
Opening existing database 'C:\BoSSS-Databases\WIP-k-LinslvPerfPar'.


In [5]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,RuntimeLocation,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,Password,ServerName,PrivateKeyFilePath,AdditionalBatchCommands,ExecutionTime,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,MonoDebug
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\flori\AppData\Local\BoSSS-LocalJobs,False,<null>,LocalPC,dotnet,<null>,"[ C:\BoSSS-Databases, C:\Users\flori ]",,,,,,,,,,
1,BoSSS.Application.BoSSSpad.SlurmClient,X:\jenkins\deploy,False,linux/amd64-openmpi,Lb2-specialPrj-Jenkins,dotnet,,[ X:\jenkins\databases == /work/scratch/fk69umer/jenkins/databases ],fk69umer,<null>,lcluster16.hrz.tu-darmstadt.de,C:\Users\flori\.ssh\id_rsa,"[ #SBATCH -C avx512, #SBATCH --mem-per-cpu=8000 ]",05:00:00,/work/scratch/fk69umer/jenkins/deploy,special00006,<null>,False


## Utility definitions

In [6]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
}

## Init grids and save to database

In [7]:
wmg.Grids

Opening existing database 'X:\jenkins\databases\WIP-k-LinslvPerfPar'.


#0: { Guid = 4276c806-f69c-43e1-86ef-1ce920cdc92b; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J10485760; Cell Count = 10485760; Dim = 3 }
#1: { Guid = 15ba8bcc-a9f1-4cb7-a605-f30461111732; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J1310720; Cell Count = 1310720; Dim = 3 }
#2: { Guid = a978fef8-2202-4332-acb3-ef6cd17ffb0b; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J552960; Cell Count = 552960; Dim = 3 }
#3: { Guid = ea4a0f29-66d2-492c-b11f-aa95bb86c947; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J163840; Cell Count = 163840; Dim = 3 }
#4: { Guid = 26992ede-1732-4424-a72f-153ea7b6f643; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J69120; Cell Count = 69120; Dim = 3 }
#5: { Guid = 81c39bfd-bc9a-4fbc-af54-bbd1a581d37a; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J20480; Cell Count = 20480; Dim = 3 }
#6: { Guid = 9a2c6da0-49a0-4574-ac5d-1c6228038000; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J2560; Cell Count = 2560; Dim = 3 }
#7: { Guid = 84cadb65-7876-441a-b4d3-017dd01d62f9; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J3

Create meshes in various resolutions:
- domain $\Omega = (0,10) \times (-1,1) \times (-1,1)$; 
- a Dirichlet boundary is set for $x = 0$, everywhere else Neumann boundaries are assumed; 
  these are typically more challenging for a linear solver than Dirichlet boundaries.
- in order to make the problem a bit challenging, a non-equidistant spacing for $y$ and $z$ coordinate are used
- the domain shape is reminiscent of a typical grid for the simulation of channel flows, with exception of the boundary conditions

In [ ]:
int[] Resolutions_3D = new int[] { 4, 8, 16, 24, 32, 48, 64, 128, 256 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];

for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(0, 10, Res*5 + 1);
    double[] yNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    double[] zNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = wmg.CurrentProject + "-SIP_Poisson_J" + J;
    
    //grids[cnt] = null;
    //foreach(IGridInfo grd in wmg.Grids) { 
    //    bool check = grd.Name.Contains(string.Format("_J"+ J));
    //    if(check) {
    //      grids[cnt] = grd;
    //    }
    //}
    
    Console.WriteLine("Creating Grid: " + GridName);
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(GridName)); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null) { // 
        Console.WriteLine($"Creating grid with {J} = {Res*5}x{Res}x{Res} cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
    
        g.DefineEdgeTags(delegate (double[] X) {
            if(Math.Abs(X[0] - 0.0) <= 1.0e-6)
                return BoundaryType.Dirichlet.ToString();
            else
                return BoundaryType.Neumann.ToString();
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J320
Found Grid: { Guid = 84cadb65-7876-441a-b4d3-017dd01d62f9; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J320; Cell Count = 320; Dim = 3 }
Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J2560
Found Grid: { Guid = 9a2c6da0-49a0-4574-ac5d-1c6228038000; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J2560; Cell Count = 2560; Dim = 3 }
Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J20480
Found Grid: { Guid = 81c39bfd-bc9a-4fbc-af54-bbd1a581d37a; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J20480; Cell Count = 20480; Dim = 3 }
Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J69120
Found Grid: { Guid = 26992ede-1732-4424-a72f-153ea7b6f643; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J69120; Cell Count = 69120; Dim = 3 }
Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J163840
Found Grid: { Guid = ea4a0f29-66d2-492c-b11f-aa95bb86c947; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J163840; Cell Count = 163840; Dim = 3 }
Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J552960
F

In [ ]:
grids

## Setup Control Object for a Solver Run

### Values/Formulas for the Values of RHS and Boundary Conditions

In [ ]:
string nl = System.Environment.NewLine;
string math_expressions_code = 
"static class InitialValues { " +
"    public static double RHS(double[] X) { " + nl +
"        return -Math.Sin(X[0]); " + nl +
"    } " + nl +
"     " + nl +
"    public static double DirichletBC(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"     " + nl +
"    public static double NeumannBC(double[] X) { " + nl +
"       if(Math.Abs(X[1] - 1.0) < 1.0e-8 || Math.Abs(X[1] + 1.0) < 1.0e-8) " + nl +
"           return 0; " + nl +
"       if(X.Length > 2 && (Math.Abs(X[2] - 1.0) < 1.0e-8 || Math.Abs(X[2] + 1.0) < 1.0e-8)) " + nl +
"           return 0; " + nl +
"     " + nl +
"       return Math.Cos(10.0); " + nl +
"   } " + nl +
"} " + nl;

In [ ]:
var RHS_value = new Formula("InitialValues.RHS", false, math_expressions_code);
var NeumannBC_value = new Formula("InitialValues.NeumannBC", false, math_expressions_code);
var DirichletBC_value = new ConstantValue(0.0);

Brief test of the boundary values:

In [ ]:
RHS_value.Evaluate(new Vector(0.2, 0.1), 0)

In [ ]:
NeumannBC_value.Evaluate(new Vector(0.2, 0.1), 0)

### Setup of Parameter Study

Polynomial degrees to test:

In [ ]:
//int[] PolyDegS = new int[] {2, 3, 4, 5, 6};
int[] PolyDegS = new int[] {2, 3, 5}; // reduced options for dev.

Solvers which we want to instrument:

In [ ]:
using BoSSS.Solution.AdvancedSolvers;

In [ ]:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.exp_Kcycle_schwarz
}; 

Number of processors:

In [ ]:
int[] MPIsizes = new int[] { 1, 2, 4, 8, 16, 32, 64, 128 };

Loop over all combinations of parameters and define a control object for each combo:

In [ ]:
var controls = new List<(SipControl ctrl, int NoOfProcs)>();
LinearSolverCode solver_name = LinearSolverCode.exp_Kcycle_schwarz;
foreach(int k in PolyDegS) {   
foreach(IGridInfo grd in grids) {
foreach(int MPIsize in MPIsizes) {
    int Np = Utils.Np(k);
    int J  = grd.NumberOfCells;
    if(J / MPIsize < 16) {
        // less than 16 cells per processor - to low for a multigrid.
        continue; 
    }
    if(J*Np/MPIsize > 500000) {
        // not interested in doing more then 500'000 DOFs per processor
        continue;
    } 
    var ctrl = new SipControl();
    controls.Add((ctrl, MPIsize));
       
    
    string caseName = string.Format("Poisson_J{0}_k{1}_Sz{2}", J, k, MPIsize);
    Console.WriteLine("setting up: " + caseName);
    ctrl.SessionName = "SIP_" + caseName;

    ctrl.SetGrid(grd);
    ctrl.savetodb = true;
    ctrl.SetDGdegree(k);

    ctrl.LinearSolver           = solver_name.GetConfig();
    var isc = ctrl.LinearSolver as IterativeSolverConfig;
    if(isc != null) {
        //Console.WriteLine(isc.ConvergenceCriterion);
        //ctrl.LinearSolver.TargetBlockSize      = Math.Min(J*Np-1,10000);
        isc.ConvergenceCriterion = 1e-8;
    }
        
        
    ctrl.InitialValues.Add("RHS", RHS_value);
    ctrl.AddBoundaryValue(BoundaryType.Dirichlet.ToString(), "T", DirichletBC_value);
    ctrl.AddBoundaryValue(BoundaryType.Neumann.ToString(), "T", NeumannBC_value);
    
    ctrl.TracingNamespaces = "*";
}
}
}

In [ ]:
//string path = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\ipPoisson\bin\Release\net5.0\PoissonBench";
//foreach(var ctrl in controls) {
//    ctrl.savetodb = false;
//    ctrl.SaveToFile(System.IO.Path.Combine(path, ctrl.SessionName + ".obj"));
//}

In [ ]:
controls.Count()

## Launch Jobs

Use the default queue defined on this machine:

In [ ]:
ExecutionQueues

In [ ]:
var myBatch = GetDefaultQueue();
myBatch

In [ ]:
foreach((var ctrl, int MPIsize) in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName);
    var j = ctrl.CreateJob();
    j.RetryCount = 1;
    j.NumberOfMPIProcs = MPIsize;
    j.Activate(myBatch);
    //ctrl.RunBatch(); // run in the default queue for this project
}

In [ ]:
//foreach(var j in wmg.AllJobs.Values)
//    j.DeleteOldDeploymentsAndSessions();

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

In [ ]:
int succJob = wmg.AllJobs.Values.Where(job => job.Status == JobStatus.FinishedSuccessful).Count();
int succSess = wmg.Sessions.Where(Si => Si.Name.Contains("SIP_Poisson") && (Si.SuccessfulTermination == true)).Count();
(succJob, succSess)

In [ ]:
int failJob = wmg.AllJobs.Values.Where(job => job.Status != JobStatus.FinishedSuccessful).Count();
int failSess = wmg.Sessions.Where(Si => Si.Name.Contains("SIP_Poisson") && (Si.SuccessfulTermination == false)).Count();
(failJob, failSess)

In [ ]:
NUnit.Framework.Assert.AreEqual(succJob + failJob, controls.Count);
NUnit.Framework.Assert.AreEqual(succJob, succSess);
NUnit.Framework.Assert.AreEqual(failJob, failSess);

In [ ]:
wmg.Sessions.Where(sess => sess.Name.StartsWith("SIP_Poisson_J"))

In [ ]:
//controls.Select(tuple => tuple.ctrl)
//        .Select(ctrl => (ctrl,ctrl.GetJob()))
//        .Select(ttt => (ttt.Item1.SessionName == ttt.Item2.Name, ttt.Item1.SessionName, ttt.Item2.Name))

In [ ]:
var NoSuccess = wmg.AllJobs.Values
                        .Where(job => job.Status != JobStatus.FinishedSuccessful)
                        .ToArray();
NoSuccess

In [ ]:
wmg.AllJobs.Values
        .Where(job => job.Name.Contains("SIP_Poisson_J552960_k2"))
        .Select(job => (job.Name, job.Status, job.LatestDeployment?.DeploymentDirectory?.FullName))
        .ToArray()

In [ ]:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail + ":  @" + ((fail.LatestDeployment?.DeploymentDirectory?.FullName) ?? " no deployment directory"));

In [ ]:
/*
string PathOffset = @"C:\Users\jenkinsci\Desktop\LinSlvPerfFail-20apr22";
foreach(var fail in NoSuccess) {
    var C = fail.GetControl();
    C.savetodb = false;
    C.SaveToFile(System.IO.Path.Combine(PathOffset, fail.Name + ".obj"));
    
    string Stdout = fail.Stdout;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stdout.txt"), Stdout);
    
    string Stderr = fail.Stderr;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stderr.txt"), Stderr);
}
*/

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("SIP_Poisson") &&
                                        (Si.SuccessfulTermination == false
                                        || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0));
FailedSessions

In [ ]:
//foreach(var s in FailedSessions)
//    s.Delete(true);

In [ ]:
//wmg.AllJobs["SIP_Poisson_J2560_k5_pMultigrid"].AllDeployments

In [ ]:
NUnit.Framework.Assert.Zero(NoSuccess.Count(), "Some Jobs Failed");

In [ ]:
//foreach(Job jF in NoSuccess)
//   jF.DeleteOldDeploymentsAndSessions();

In [ ]:
NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not derminate successfully.");

List Output of some job (arbitrarily the first one):

In [ ]:
wmg.AllJobs.First().Value.Stdout

## Create Table for Post-Processing

(moved to post-processing worksheet)

In [ ]:
/*
// evaluators to add additional columns to the session table
static class AddCols {
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("MatrixAssembly");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Aggregation_basis_init");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Init");
        var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Run");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
        }
}
*/

In [ ]:
/*
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);
*/

In [ ]:
/*
var FullSessTab = wmg.SessionTable;

// The Session column can't be serialized,
// we have to remove it
List<string> AllCols = FullSessTab.GetColumnNames().ToList();
AllCols.Remove("Session");
FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());
*/

In [ ]:
//foreach(var s in AllCols) Console.WriteLine(s);

In [ ]:
//foreach(var s in wmg.Sessions[0].KeysAndQueries.Keys) Console.WriteLine(s + " \t = " + wmg.Sessions[0].KeysAndQueries[s]);

In [ ]:
/*var SubTab = FullSessTab.ExtractColumns(
    "SessionName","DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", "MatrixAssembly",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "NoIter");
*/

In [ ]:
//SubTab.Print();

In [ ]:
/*
// Filename
var now               = DateTime.Now;
FullSessTab.TableName = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
string docpath        = FullSessTab.TableName + ".json";
SubTab.SaveToFile(wmg.CurrentProject + ".json");
SubTab.ToCSVFile(wmg.CurrentProject + ".csv");
*/